# Section 5.5.2
# Animate head in strip of land bounded by surface water at both sides in which after a sudden head change occurred

Use the solution for the decline of head in a strip of land within parallel ditches to animat it.

Animation is done with `matplotlib.anamation.FuncAnimation`. This requires two functions. One that initializes the lines to be drawn and the other to update its data in an iteration loop. The latter function takes a single integer input that is the frame number within the iteration.

It makes sense to define a class, in this case "Strip" which has a method to compute itself. Doing this, the class is instantiated with all its data before running the animation, so that the aninamtion only requires the counter.

@TO 2020-12-09

In [4]:
attribs = lambda obj: [o for o in dir(obj) if not o.startswith('_')]

import numpy as np
from scipy.special import erfc
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import pdb


def newfig(title='title', xlabel='xlabel', ylabel='ylabel',
                   xlim=None, ylim=None, size_inches=(8, 6)):
    fig, ax = plt.subplots()
    fig.set_size_inches(size_inches)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    if xlim:
        ax.set_xlim(xlim)
    if ylim:
        ax.set_ylim(ylim)
    ax.grid()
    return fig, ax

### Strip formula

$$s(x, t) = h_{left} \sum_{i=1}^\infty 
\left\{
\mbox{erfc}\left[\left\{\left(2 i - 1\right) L + \left(x - \frac L 2 \right)\right\}\sqrt{\frac S {4 kD t}}\right]
-
\mbox{erfc}\left[
              \left\{\left(2 i - 1 \right) L - \left(x - \frac L 2\right)\right\}\sqrt{\frac S {4 kD t}}\right]
\right\}
+
h_{right} \sum_{i=1}^\infty 
\left\{
      \mbox{erfc}\left[\left\{\left(2 i - 1 \right) L - \left(x + \frac L 2 \right)\right\}\sqrt{\frac S {4 kD t}}\right]
-
\mbox{erfc}\left[
          \left\{\left(2 i - 1 \right) L + \left(x + \frac L 2 \right)\right\}\sqrt{\frac S {4 kD t}}\right]
\right\}
$$

In [18]:
# First set up the figure, the axis, and the plot element we want to animate
class Strip:
    
    def __init__(self, b=None, hL=None, hR=None, kD=None, S=None, n=20):
        """Return a strip object.
        
        Parameters
        ----------
        b: float
            half-width of the strip
        hL: float
            initial head above drainage base at left of strip
        hR: float
            initial head above drainage base at right of strip
        kD: float
            transmissivity
        S: float
            Storage coefficient
        n: number of points in the half-strip
        """
        self.hL = hL
        self.hR = hR
        self.kD = kD
        self.S = S
        self.b = b
        self.n = n
        self.x = np.linspace(-self.b, self.b, n + 1)
        self.tchar = self.b ** 2 * self.S / self.kD
        self.T50 = (2 / np.pi) ** 2 * np.log(2) * self.tchar
        
    def compute(self, t, n=20):
        root = np.sqrt(self.S / (4 * self.kD * t))
        s = np.zeros_like(self.x)
        for i in range(1, n + 1):
            pif = (2 * i - 1) * self.b * 2
            xmb = self.x - self.b
            xpb = self.x + self.b
            s += self.hL * (
                    erfc((pif + xmb) * root) - erfc((pif - xmb) * root)) +\
                 self.hR * (
                    erfc((pif - xpb) * root) - erfc((pif + xpb) * root))
        return s
    
    def plot(self, times=None, n=20, ax=None):
        """Plot the head for all times."""
        for t in times:
            s = self.compute(t)
            ax.plot(self.x, s, label='t= {:.3f} d'.format(t))


In [27]:
data = {'kD': 600, 'S':0.2, "hL":2, "hR":1, "b": 150, "n":20}

strip = Strip(**data)

title = "Animation of head in infinite half strip with initial head increase at x=0.\n" +\
        'kD={:.0f} m2/d, S={:.2f}, b={:.0f} m, hL={:.2f} m, hR={:.2f} m'\
        .format(*[data[k] for k in ['kD', 'S', 'b', 'hL', 'hR']])

fig, ax = newfig(title, "x [m]", "head / water table [m]",
                  xlim=(-data['b'], data['b']),
                  ylim=(min(0, min(data['hL'], data['hR'])), max(data['hL'], data['hR'])),
                  size_inches=(12, 8))

line, = ax.plot([], [], lw=2)
text  = ax.text(0.05, 0.90, '', fontsize=14, transform=ax.transAxes)
xchar = 50
ax.text(0.05, 0.8, 'Head change, tchar = {:.3f} d, T50 = {:.3f} d'
                    .format(strip.tchar, strip.T50),
                    fontsize=14, transform=ax.transAxes)

n = 6  # total time 6 times the halftime
m = 30 # subdivision of each halftime
T50_times = strip.T50 * np.arange(n)
times    = strip.T50 * np.arange(0, n, 1/m)[1:]

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return line, # receiver expects a tuple of the artists involved

# animation function.  This is called sequentially
def animate(t, strip, t50times, ax):
    # global declaration not strictly needed, but nice to make clear
    global line, text
    T50 = strip.T50
    y = strip.compute(t)
    line.set_data(strip.x, y)
    text.set_text('time = {:8.3f} d, t_rel = {:.1f}'.format(t, t/T50))
    
    # Leave line behind at every tchar multiple.
    if t in T50_times:
        ax.plot(strip.x, strip.compute(t), label='t / T50 = {:.1f}'.format(t / T50))
    if t == T50_times[-1]:
        ax.legend()
    return line, # receiver expects a tuple of the artitsts involved

# call the animator.  blit=True means only re-draw the parts that have changed.
print("Patience, computing and generating video takes about 30 sec. on mac...")
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=times, fargs=(strip, T50_times, ax),
                               interval=20, blit=True, repeat=False)

plt.close(anim._fig)
out = HTML(anim.to_html5_video())
#out

Patience, computing and generating video takes about 30 sec. on mac...


In [28]:
# save the animation as an mp4.  This requires ffmpeg or mencoder to be
# installed.  The extra_args ensure that the x264 codec is used, so that
# the video can be embedded in html5.  You may need to adjust this for
# your system: for more information, see
# http://matplotlib.sourceforge.net/api/animation_api.html
anim.save('Land_strip_animation.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
print(anim.save_count, " frames saved.")

179  frames saved.


In [29]:
out